In [2]:
import sys
sys.path.append('../')
from core.derived import create_predictor

# enable debug mode
from core.util import GlobalSettings
gs = GlobalSettings()
gs.debug = True

# load model config
run = '20240607000000'
model = 'weights_e30_final_001'
predictor = create_predictor(run, model)

Debug mode is off. Using settings.json.
Debug mode is on. Using settings_ThinkPadP16.json.
Imported embedding from D:/Drive/Project/Hematology/derived\20240607000000_AdamDO0.2D0FzBn\embedding_weights_e30_final.h5
Fitted label classifier with 16591 samples.


In [3]:
import os
from PIL import Image

# load image
file = os.path.join(gs.get('data_folder'), 'PBC_dataset_normal_DIB\lymphocyte\LY_392666.jpg')
# compute features
features = predictor.get_feature(file)
print(features.shape)

1/1 [==============================] - 8s 8s/step
(1280,)


In [4]:
y_pred_label, y_pred_code = predictor.predict(file)
print(y_pred_label)

1/1 [==============================] - 0s 21ms/step
lymphocyte


In [5]:
y_pred_label, y_pred_code, prob, tsne_embedding_query = predictor.predict_and_reference(file)
print(y_pred_label)
print(prob)

1/1 [==============================] - 0s 22ms/step
['lymphocyte', 'basophil', 'monocyte', 'erythroblast', 'platelet', 'ig', 'neutrophil', 'eosinophil', 'blast']
[0.9952147403530458, 0.002269279902900542, 0.0010451136778836113, 0.00045702377111632577, 0.00036362305923974867, 0.00033751215124323044, 0.00013978196169513158, 0.00011591736364020423, 5.7007759235736324e-05]


In [6]:
from model.loader import load_loader

opt = {'file_list': os.path.join(gs.get('data_folder'), '20230926_DIB.csv'), 
       'split_seed': 0, 
       'test_ratio': 0.1, 
       'kfold': 5, 
       'i_kfold': 2}
loader = load_loader(opt)
test_df = loader.test_df
predictor.set_test(test_df)
predictor.val_index = predictor._test_df.index[[0,1]]
predictor.compute_feature()
y_pred_code, y_true_code = predictor.validate(category='label')
print(test_df.iloc[[0,1]]['label'].tolist())
print(y_true_code)
print(y_pred_code)

Split 23044 images to 20739 train and 2305 test images
Split 20739 images to 16591 train and 4148 validation images
1/1 [==============================] - 0s 292ms/step
['blast', 'neutrophil']
[1 7]
[1 7]


Supervised

In [5]:
import sys
sys.path.append('../')
import os
# enable debug mode
from core.util import GlobalSettings
gs = GlobalSettings()
gs.debug = True
from model.loader import load_loader

opt = {'file_list': os.path.join(gs.get('data_folder'), '20230926_DIB.csv'), 
       'split_seed': 0, 
       'test_ratio': 0.1, 
       'kfold': 5, 
       'i_kfold': 2}
loader = load_loader(opt)
train_df = loader.train_df
val_df = loader.val_df

Debug mode is off. Using settings.json.
Debug mode is on. Using settings_ThinkPadP16.json.
Split 23044 images to 20739 train and 2305 test images
Split 20739 images to 16591 train and 4148 validation images


In [6]:
import core.derived
from model.predictor import SiamesePredictor
from model.model import load_model

run = '20240530152917'
model = 'weights_e30_final_001'

opt = core.derived.get_run_config(run)
model_opt = opt['model']
model_class = load_model(model_opt)

predictor = SiamesePredictor(model=model_class,
                            train=train_df, 
                            test=val_df, 
                            support_index=None, 
                            val_index=val_df.index.values)

In [7]:
predictor.support_index = train_df.index.tolist()
predictor.compute_feature()

130/130 [==============================] - 3s 26ms/step


In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

category = 'label'
pipeline = make_pipeline(StandardScaler(), 
                        SVC(kernel=opt.get('kernel', 'linear'),
                            C=opt.get('C', 1.0),
                            decision_function_shape='ovr',
                            probability=True, 
                            random_state=0))
predictor.set_classifier(pipeline, category)
predictor.fit(category)
y_pred_code, y_true_code = predictor.validate(category)

Fitted label classifier with 16591 samples.


In [ ]:
from core.derived import create_predictor

# load model config
run = '20240530152917'
model = 'weights_e30_final_001'
predictor = create_predictor(run, model)